![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=6> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=5> Óptica Digital, curso 2022-2023 </font>
</div>

    
<div align="center"> 
<font size=5> Tema 9 - Ejemplo de optimización y funciones de mérito</font>
</div>


**Jesús del Hoyo Muñoz**

# Objetivos

    • Aprender a utilizar el módulo *optimize* de *scipy*.
    • Ver el ejemplo de la implementación de una función de mérito.

# Importar modulos

Vamos a cargar los módulos y las clases que necesitamos.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams['figure.dpi']=150
rcParams['image.cmap']='hot'

from py_pol.jones_vector import Jones_vector
from py_pol.jones_matrix import Jones_matrix, degrees

from scipy.optimize import least_squares

In [ ]:
# Cargar datos del SLM
data = np.load("SLM_Jones_components.npz")
components = [data["J0"] * np.exp(1j * data["d0"]),
             data["J1"] * np.exp(1j * data["d1"]),
             data["J2"] * np.exp(1j * data["d2"]),
             data["J3"] * np.exp(1j * data["d3"])]
Jslm = Jones_matrix().from_components(components);

---
# 1. Target

Vamos a definir una función de mérito y a hacer una optimización empleando una distribución objetivo que queremos obtener.

La modulación que queremos conseguir es mixta: cambio lineal en amplitud y fase.

In [ ]:
def system_transmition(angles, Jslm:Jones_matrix):
    (a1, a2, a3, a4) = angles
    Ein = Jones_vector().circular_light(intensity=2) # intensity 2, since circular -> perfect diatenuator will remove half of it
    Jp1 = Jones_matrix().diattenuator_perfect(azimuth=a1)
    Jr1 = Jones_matrix().quarter_waveplate(azimuth=a2)
    Jr2 = Jones_matrix().quarter_waveplate(azimuth=a2)
    Jp2 = Jones_matrix().diattenuator_perfect(azimuth=a2)

    Efin = Jp2 * Jr2 * Jslm * Jr1 * Ein
    
    Ifin, phase = Efin.parameters.intensity(), Efin.parameters.global_phase()

    return Ifin, phase

    

In [ ]:
def score(angles, Jslm):
    # Ideal transmission
    v = np.linspace(0, Jslm.size - 1, Jslm.size)
    t = v / v.max()
    t_phase = 2 * np.pi * v.max()

    # System transmition
    t_i_transmission, t_phase_transmission = system_transmition(angles, Jslm)

    t_phase = 2 * np.pi * v.max()
    sqr_error = (t_i_transmission- t) ** 2 + (t_phase_transmission - t_phase) ** 2
    return sqr_error

In [ ]:
init = np.random.rand(4) * np.pi / 2  # angles are given in radians.
bounds = (np.zeros(4), np.ones(4) * np.pi)
result = least_squares(score, x0=init, bounds=bounds, args= [Jslm])


In [ ]:
from scipy.optimize import minimize

def system_transmition_pol(angles, Jslm:Jones_matrix):
    (a1, a2, a3, a4) = angles
    Ein = Jones_vector().circular_light(intensity=2) # intensity 2, since circular -> perfect diatenuator will remove half of it
    Jp1 = Jones_matrix().diattenuator_perfect(azimuth=a1)
    Jr1 = Jones_matrix().quarter_waveplate(azimuth=a2)
    Jr2 = Jones_matrix().quarter_waveplate(azimuth=a2)
    Jp2 = Jones_matrix().diattenuator_perfect(azimuth=a2)

    Efin = Jp2 * Jr2 * Jslm * Jr1 * Ein
    Ifin, phase = Efin.parameters.intensity(), Efin.parameters.global_phase()
    az, el = Efin.parameters.

    return Ifin, phase, az, el

def score_pol(angles, Jslm):
    # Ideal transmission
    v = np.linspace(0, Jslm.size - 1, Jslm.size)
    t = v / v.max()
    t_phase = 2* np.pi * v.max()

    # System transmition
    t_samp, t_phase,t_az, t_el = system_transmition_pol(angles, Jslm)

    t_phase = 2 * np.pi * v.max()
    error = np.sum(t_el**2) + 1/(t_az.max() - t_az.min()) + (t_samp.max() - t_samp.min()) + 1 / t_samp.mean()
    return error



# YOU ARE MISSING A LOT
result = minimize(score_pol)

## Keep playing around until you find something that works.


---
# Parámetros

En este caso, definiremos la función de mérito usando unos parámetros adecuados. 

En este caso vamos a buscar una distribución diferente: Polarización lineal con variación lineal del azimut lo más grande posible.

In [ ]:
from functools import partial

partial()

In [ ]:
def score_pol_constnat(angles, Jslm, weights):
    # Ideal transmission
    v = np.linspace(0, Jslm.size - 1, Jslm.size)
    t = v / v.max()
    t_phase = 2 * np.pi * v.max()

    # System transmition
    t_samp, t_phase, t_az, t_el = system_transmition_pol(angles, Jslm)

    (w0, w1, w2, w3,w4, ) = weights

    t_phase = 2 * np.pi * v.max()
    error = (
        w0 * np.sum(t_el**2)
        + w1 * 1 / (t_az.max() - t_az.min())
        + w3 * np.std(np.diff(t_az))
        + w4 * np.std(t_az)
        + w5 * 1 / t_samp.mean()
    )
    return error


# YOU ARE MISSING A LOT
result = minimize(score_pol)
